In [1]:
# imports of python libraries
import requests
import os
import pandas as pd
import json

In [3]:
# import citybike dataframe 
import pandas as pd
citybike_df= pd.read_csv(
                        f"/Users/tinapham/Desktop/lighthouse-data-notes/ProJect/LHL_StatsModel_PythonProject/data/citybike.csv", 
                        sep=","
                        )
citybike_df

,Station_name,Available_bikes,Longitude,Latitude
0,Cégep Marie-Victorin,10,-73.606011,45.617500
1,Gare d'autocars de Montréal (Berri / Ontario),6,-73.564257,45.516926
2,Molson / William-Tremblay,6,-73.565012,45.541549
3,Ateliers municipaux de St-Laurent (Cavendish /...,11,-73.711186,45.506176
4,Place Rodolphe-Rousseau (Gohier / Édouard-Laurin),7,-73.682498,45.512994
...,...,...,...,...
757,Jean-Talon / de l'Épée,14,-73.624063,45.527487
758,Gare LaSalle (Highlands),4,-73.657136,45.425613
759,Métro St-Michel (Shaughnessy / St-Michel),2,-73.599488,45.559076
760,St-Hubert / René-Levesque,4,-73.556487,45.514369


In [5]:
# Set API KEY with user's environment variables
import os
FOUSQUARE_API_KEY   =os.environ["FOURSQUARE_API_KEY2"]
YELP_API_KEY        =os.environ["YELP_API_KEY"]

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests
# NOTE: API get request only accept latitude and longitude as a string
# Create a list of latitudes and longitudes from the citybike dataframe to pass through a for loop
latitudes = citybike_df['Latitude'].tolist()
longitudes = citybike_df['Longitude'].tolist()

# Create an empty dictionary to store the json object from the API requist
list_4square0=[]

# Create a for loop that iterates over the list of (latitudes, longitudes) which can then be use to loop into a request url 
for latitude, longitude in zip(latitudes, longitudes):
    
    # Create a Foursquare API request within the for loop to iterates response over each pair of (latitudes, longitudes)
    resquest_4square0 = requests.get(
        "https://api.foursquare.com/v3/places/search",
        params={
            "query": ["bar","restaurant","park","museum","market","grocery"],
            "ll": f"{latitude},{longitude}",
            "radius": 1000,
            "limit": 50,
        },
        headers= {
                "Accept": "application/json",
                "Authorization": FOUSQUARE_API_KEY
                }
    )
    list_4square0.append(resquest_4square0.json())
    # Create a dictionary key with 'latitude_longitude' 
    #json_4square_key0= f'{latitude}_{longitude}'
    
    # Passing all the response json object into the dictionary using 'latitude_longitude' as key
    #dict_json_4square0[json_4square_key0]=response_4square0.json()

In [ ]:
# QA: number of key in the 4square response dictionary should match with number of row in citybike
response_dict=len(list_4square0.keys())
city_bike_df= citybike_df['Longitude'].count()
if response_dict==city_bike_df:
    print('Match')
else:
    print('Fail')

In [ ]:
import json
# Save response as .json file to limit amount of API request
# with open("response_4square_list0.json", "w") as outfile:
    #json.dump(list_json_4square0, outfile)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Testing Obtain POI Name
list_4square0[0]['results'][0]['name']
# Json structure list--> result--> list-->'name'

In [ ]:
# Empty list to contain all iterate object from the for loop
list_4squareb=[]

# Create the first for loop to extract data from results
for i in range(len(list_4square0)):
    list_4square_a=(list_4square0[i]['results'])
    
    # Create the second for loop to get data inside the list
    for n in range(len(list_4square_a)):
        list_4squareb.append(list_4square_a[n])

# Parse list into a dataframe
raw_4square_df=pd.json_normalize( list_4squareb)

# Drop duplicate row from 'fsq_id'
df_4square= raw_4square_df.drop_duplicates(subset='fsq_id')

Put your parsed results into a DataFrame

In [ ]:
# Create an empty dictionary with key as the column for dataframe
Clean_4square_dict={"POI_Name":[],
                "POI_distance":[]}

# Insert data into each column
Clean_4square_dict['POI_Name']=df_4square['name']
Clean_4square_dict['POI_distance']=df_4square['distance'].astype(int)
Clean_4square_dict['POI_categories']= df_4square['categories']
Clean_4square_dict['POI_Latitude']=df_4square['geocodes.main.latitude'].astype(float)
Clean_4square_dict['POI_Longitude']=df_4square['geocodes.main.longitude'].astype(float)

# Turn ditionary into a dataframe
Clean_4square_df= pd.DataFrame(Clean_4square_dict)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests
import json
list_yelp0=[]
for item, series in citybike_df.iterrows():
    lat = series['Latitude']
    long = series ['Longitude']
    cat=["localflavor","landmark","park", "restaurant","bar"]
    url_yelp = "https://api.yelp.com/v3/businesses/search"
    params= {"latitude": lat,
        "longitude": long,
        "categories": cat,
        "radius": 1000,
        "limit": 50}
    headers = {
            "accept": "application/json",
            "Authorization": 'Bearer %s' % YELP_API_KEY
            }
    response_yelp = requests.get(url_yelp, headers=headers, params=params)
    data_yelp=response_yelp.json()
    # NOTE: Yelp limit to 500 call per day, use range to stop the call at 500
    #for i in range(0,499):
    list_yelp0.append(data_yelp)

In [ ]:
# Save data to limit API call
# with open("response_yelp_list_raw.json", "w") as outfile:
    #json.dump(list_yelp0, outfile)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Empty list to contain all iterate object from the for loop
list_yelp_b=[]

# Create the first for loop to extract data from list:
for i in range(0,499):
    list_yelp_a=(list_yelp0[i])
    
    # Create the second for loop to get data inside the parents directory businesses
    for n in range(len(list_yelp_a['businesses'])):
        list_yelp_b.append(list_yelp_a['businesses'][n])

# Parse list into a dataframe
raw_yelp_df=pd.json_normalize( list_yelp_b)

# Drop duplicate row from 'id'
#df_yelp= raw_yelp_df.drop_duplicates(subset='id')

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provided more complete data. Yelp provided distance from each bike station, rating for POI, review count and details about each business. Foursquare does has rating on their normal website, however, maybe it is not provided due to data integrity.

Get the top 10 restaurants according to their rating

#### Joining data to citybike_df

In [ ]:
# Create a for loop to count for POI belong to each bike station from Foursquare
POI_4square=[]
# Parsing data from results
for i in list_4square0:
    POI_4square.append(i['results'])
# Count for children from the parents results
Count_4square_POI=[]
for n in range(len(POI_4square)):
    Count_4square_POI.append(len(POI_4square[n]))
   

In [ ]:
# Getting Yelp POI count
POI_Yelp=[]
# Parsing json object from businesses
for i in range(len(list_yelp0)):
    POI_Yelp.append(list_yelp0[i]['businesses'])
    
# Count for children from the parents results
Count_Yelp_POI=[]
for n in range(len(POI_Yelp)):
    Count_Yelp_POI.append(len(POI_Yelp[n]))

In [ ]:
# Joining the data to citybike_df
citybike_df["Count_4square_POI"]=Count_4square_POI
citybike_df["Count_Yelp_POI"]=Count_Yelp_POI

In [ ]:
# Save the dataframe for later usage
citybike_df.to_csv(r'/Users/tinapham/Desktop/lighthouse-data-notes/ProJect/LHL_StatsModel_PythonProject/data/joindf_clean.cvs', index=False)